## __Single gene analysis__

```GP2 ❤️ Open Science 😍```

### Description

Using "individual level data"....:

__0. Getting Started__
- Loading Python libraries
- Defining functions
- Installing packages

__1. Copy data from workspace to cloud environment__

__2. Extract PTPA__

__3. Annotate PTPA variants__

__4. Extract coding/non-syn variants__

__5. Calculate frequency in cases versus controls__

__6. Calculate frequency (homozygotes)in cases versus controls__

__7. Save out results__



## Getting started

### Loading Python libraries

In [2]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

### Defining functions

In [3]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [4]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v3.0
## Explicitly define release v3.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v3.0')
print(f'Path to GP2 v3.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v3.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v3.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print('')

Billing and Workspace
Workspace Name: PP2A
Billing Project: terra-02e7cd23
Workspace Bucket, where you can upload and download data: gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0

GP2 v3.0
Path to GP2 v3.0 Clinical Data: gs://gp2tier2/release3_31102022/clinical_data
Path to GP2 v3.0 Raw Genotype Data: gs://gp2tier2/release3_31102022/raw_genotypes
Path to GP2 v3.0 Imputed Genotype Data: gs://gp2tier2/release3_31102022/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink



### Install packages

### Install Plink 1.9 and Plink 2.0

In [5]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [6]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


### Remote restrictions

In [7]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [8]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [9]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [10]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


### Install ANNOVAR: Download sources of annotation

In [11]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/


NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [12]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## Copy data from workspace to cloud environment

In [13]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/PTPA/'
shell_do(f'mkdir -p {WORK_DIR}') # f' significa f-string - contiene expresiones para ejecutar el codigo

Making a working directory


Executing: mkdir -p /home/jupyter/PTPA/


In [14]:
# Copy data
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/pheno_PPA2_100KB* {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/COVS.txt {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 -m cp gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0/pheno_PPA2_100KB* /home/jupyter/PTPA/


Copying gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0/pheno_PPA2_100KB.bed...
Copying gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0/pheno_PPA2_100KB.bim...    
Copying gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0/pheno_PPA2_100KB.fam...    
/ [3/3 files][  8.7 MiB/  8.7 MiB] 100% Done                                    
Operation completed over 3 objects/8.7 MiB.                                      


Executing: gsutil -u terra-02e7cd23 -m cp gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0/COVS.txt /home/jupyter/PTPA/


Copying gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0/COVS.txt...
/ [1/1 files][142.0 KiB/142.0 KiB] 100% Done                                    
Operation completed over 1 objects/142.0 KiB.                                    


## Extract PTPA

In [15]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
head COVS.txt

ID	SEX	PHENO	AGE_BASELINE
BF-1002	2	2	66
BF-1003	1	2	61
BF-1004	1	2	62
BF-1006	1	2	59
BF-1008	2	2	69
BF-1010	1	2	65
BF-1011	1	2	70
BF-1015	1	2	61
BF-1016	2	2	71


In [16]:
%%bash
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
head pheno_PPA2_100KB.fam

BF-1001 BF-1001 0 0 0 1
BF-1002 BF-1002 0 0 0 2
BF-1003 BF-1003 0 0 0 2
BF-1004 BF-1004 0 0 0 2
BF-1005 BF-1005 0 0 0 1
BF-1006 BF-1006 0 0 0 2
BF-1008 BF-1008 0 0 0 2
BF-1009 BF-1009 0 0 0 1
BF-1010 BF-1010 0 0 0 2
BF-1011 BF-1011 0 0 0 2


COMPHET

In [17]:

%%bash 
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

grep -e "PP-40916" -e "PP-41373" COVS.txt > AMP_PD_comphet.txt
head AMP_PD_comphet.txt

PP-40916	2	2	77
PP-41373	2	2	65


In [18]:
%%bash 
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

grep -e "BF-1163" -e "HB-PD_INVTA736CUG" -e "HB-PD_INVVT242JK8" -e "LC-180005" -e "PD-PDJH196MW5" -e "PD-PDNL129YR9" -e "PD-PDPM717VY8" -e "PD-PDTB425RD6" -e "PD-PDTG780ZLZ" -e "PD-PDUD605VWP" -e "PD-PDVG991JYT" -e "PP-3190" -e "PP-3376" -e "PP-3529" -e "SU-32089" -e "SY-PDJC327HP1" -e "SY-PDLT248VD1" COVS.txt > AMP_PD_9:129147457_hom_cases.txt
head AMP_PD_9:129147457_hom_cases.txt
tail AMP_PD_9:129147457_hom_cases.txt

BF-1163	1	2	82
PP-3190	1	2	82
PP-3376	2	2	70
PP-3529	1	2	65
HB-PD_INVTA736CUG	1	2	65
HB-PD_INVVT242JK8	1	2	78
LC-180005	2	2	85
PD-PDJH196MW5	1	2	65
PD-PDNL129YR9	2	2	71
PD-PDPM717VY8	2	2	68
PD-PDJH196MW5	1	2	65
PD-PDNL129YR9	2	2	71
PD-PDPM717VY8	2	2	68
PD-PDTB425RD6	1	2	45
PD-PDTG780ZLZ	1	2	53
PD-PDUD605VWP	1	2	71
PD-PDVG991JYT	2	2	68
SU-32089	2	2	72
SY-PDJC327HP1	1	2	37
SY-PDLT248VD1	2	2	78


In [19]:
%%bash 
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

grep -e "BF-1256" -e "HB-PD_INVLE494MFY" -e "HB-PD_INVML326LKX" -e "HB-PD_INVMX828VMK" -e "HB-PD_INVVF205THQ" -e "LB-06020" -e "LB-06113" -e "LB-06372" -e "LB-06594" -e "LB-06753" -e "LB-07306" -e "LB-07379" -e "LB-07932" -e "LC-470001" -e "PD-PDRG028AC6" -e "PP-3071" -e "PP-3264" -e "PP-40758" COVS.txt > AMP_PD_9:129147457_hom_controls.txt
head AMP_PD_9:129147457_hom_controls.txt
tail AMP_PD_9:129147457_hom_controls.txt

BF-1256	1	1	61
HB-PD_INVLE494MFY	2	1	75
HB-PD_INVML326LKX	2	1	70
HB-PD_INVMX828VMK	2	1	64
HB-PD_INVVF205THQ	2	1	58
LB-06020	2	1	68
LB-06113	2	1	65
LB-06372	1	1	61
LB-06594	2	1	76
LB-06753	1	1	86
LB-06594	2	1	76
LB-06753	1	1	86
LB-07306	1	1	75
LB-07379	1	1	67
LB-07932	1	1	85
LC-470001	2	1	58
PD-PDRG028AC6	2	1	66
PP-3071	1	1	71
PP-3264	2	1	60
PP-40758	2	1	45


In [20]:

%%bash 
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

grep -e "LB-07787" COVS.txt > AMP_PD_9:129134852_hom_controls.txt
head AMP_PD_9:129134852_hom_controls.txt
tail AMP_PD_9:129134852_hom_controls.txt

LB-07787	1	1	87
LB-07787	1	1	87


In [21]:
%%bash
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
awk '{print $1, $1, $2}' COVS.txt > temp

In [22]:
%%bash
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
head temp

ID ID SEX
BF-1002 BF-1002 2
BF-1003 BF-1003 1
BF-1004 BF-1004 1
BF-1006 BF-1006 1
BF-1008 BF-1008 2
BF-1010 BF-1010 1
BF-1011 BF-1011 1
BF-1015 BF-1015 1
BF-1016 BF-1016 2


In [23]:
%%bash
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
sed '1d' temp > sex_noheader.txt

In [24]:
%%bash
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
head sex_noheader.txt

BF-1002 BF-1002 2
BF-1003 BF-1003 1
BF-1004 BF-1004 1
BF-1006 BF-1006 1
BF-1008 BF-1008 2
BF-1010 BF-1010 1
BF-1011 BF-1011 1
BF-1015 BF-1015 1
BF-1016 BF-1016 2
BF-1031 BF-1031 2


In [25]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

# Update sex variable
/home/jupyter/tools/plink \
--bfile pheno_PPA2_100KB \
--update-sex sex_noheader.txt  \
--make-bed \
--out temp_plink

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to temp_plink.log.
Options in effect:
  --bfile pheno_PPA2_100KB
  --make-bed
  --out temp_plink
  --update-sex sex_noheader.txt

14998 MB RAM detected; reserving 7499 MB for main workspace.
3603 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to temp_plink.nosex .
6775 phenotype values loaded from .fam.
--update-sex: 6775 people updated, 1064 IDs not present.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42

In [26]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

# PTPA: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink \
--bfile temp \
--chr 9 \
--from-bp 129110950 \
--to-bp 129148946 \
--make-bed \
--out pheno_PTPA

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PTPA.log.
Options in effect:
  --bfile temp
  --chr 9
  --from-bp 129110950
  --make-bed
  --out pheno_PTPA
  --to-bp 129148946

14998 MB RAM detected; reserving 7499 MB for main workspace.
986 out of 3603 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to pheno_PTPA.nosex .
6775 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%

phenotypes to be ignored, use the --allow-no-sex flag.


### Visualize plink files bim, fam and bed

In [27]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

head pheno_PTPA.bim

9	chr9:129110951:T:C	0	129110951	C	T
9	chr9:129110955:G:C	0	129110955	C	G
9	chr9:129110977:A:C	0	129110977	C	A
9	chr9:129111023:G:C	0	129111023	C	G
9	chr9:129111036:G:C	0	129111036	C	G
9	chr9:129111059:G:A	0	129111059	A	G
9	chr9:129111157:A:T	0	129111157	T	A
9	chr9:129111178:T:C	0	129111178	C	T
9	chr9:129111181:T:C	0	129111181	C	T
9	chr9:129111184:G:C	0	129111184	C	G


In [28]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

head pheno_PTPA.fam

BF-1001 BF-1001 0 0 0 1
BF-1002 BF-1002 0 0 0 2
BF-1003 BF-1003 0 0 0 2
BF-1004 BF-1004 0 0 0 2
BF-1005 BF-1005 0 0 0 1
BF-1006 BF-1006 0 0 0 2
BF-1008 BF-1008 0 0 0 2
BF-1009 BF-1009 0 0 0 1
BF-1010 BF-1010 0 0 0 2
BF-1011 BF-1011 0 0 0 2


In [29]:
%%bash

# Visualize bed file
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

head pheno_PTPA.bed

l�������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������

In [30]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_PTPA \
--recode vcf-fid \
--out pheno_PTPA

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PTPA.log.
Options in effect:
  --bfile pheno_PTPA
  --out pheno_PTPA
  --recode vcf-fid

14998 MB RAM detected; reserving 7499 MB for main workspace.
986 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to pheno_PTPA.nosex .
6775 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57

phenotypes to be ignored, use the --allow-no-sex flag.


## Annotate PTPA variants

In [31]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_PTPA.vcf

tabix -f -p vcf pheno_PTPA.vcf.gz
tabix -f -p vcf pheno_PTPA.vcf.gz

[bgzip] can't create pheno_PTPA.vcf.gz: File exists


In [32]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_PTPA.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_PTPA.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_PTPA.vcf.gz > pheno_PTPA.annovar.avinput>
NOTICE: Finished reading 993 lines from VCF file
NOTICE: A total of 986 locus in VCF file passed QC threshold, representing 953 SNPs (670 transitions and 283 transversions) and 0 indels/substitutions
NOTICE: Finished writing allele frequencies based on 9186920 SNP genotypes (6458800 transitions and 2728120 transversions) and 0 indels/substitutions for 9640 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_PTPA.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_PTPA.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver hg3

## Extract coding and non-syn variants

In [33]:
# Visualize multianno file
PTPA = pd.read_csv(f'{WORK_DIR}/pheno_PTPA.annovar.hg38_multianno.txt', sep = '\t')
PTPA.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo9643,Otherinfo9644,Otherinfo9645,Otherinfo9646,Otherinfo9647,Otherinfo9648,Otherinfo9649,Otherinfo9650,Otherinfo9651,Otherinfo9652
0,9,129110951,129110951,T,C,UTR5,PTPA,NM_021131:c.-650T>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,9,129110955,129110955,G,C,UTR5,PTPA,NM_021131:c.-646G>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,9,129110977,129110977,A,C,UTR5,PTPA,NM_021131:c.-624A>C,.,.,...,0/0,0/0,1/1,0/0,0/1,0/1,0/1,0/1,1/1,0/0
3,9,129111023,129111023,G,C,UTR5,PTPA,NM_021131:c.-578G>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,9,129111036,129111036,G,C,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [34]:
# Filter exonic variants
coding = PTPA[PTPA['Func.refGene'] == 'exonic']
coding.count()

Chr              25
Start            25
End              25
Ref              25
Alt              25
                 ..
Otherinfo9648    25
Otherinfo9649    25
Otherinfo9650    25
Otherinfo9651    25
Otherinfo9652    25
Length: 9663, dtype: int64

In [35]:
coding.head(25)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo9643,Otherinfo9644,Otherinfo9645,Otherinfo9646,Otherinfo9647,Otherinfo9648,Otherinfo9649,Otherinfo9650,Otherinfo9651,Otherinfo9652
25,9,129111606,129111606,T,C,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon1:c.T6C:p.A2A,PTPA:NM_17...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
255,9,129120531,129120531,C,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon2:c.C50G:p.P17R,PTPA:NM_...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
256,9,129120565,129120565,G,A,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon2:c.G84A:p.K28K,PTPA:NM_...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
311,9,129123066,129123066,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001193397:exon3:c.C39T:p.Y13Y,PTPA:NM_...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
312,9,129123135,129123135,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001193397:exon3:c.C108T:p.S36S,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
420,9,129127987,129127987,A,G,exonic,PTPA,.,synonymous SNV,PTPA:NM_178001:exon4:c.A240G:p.E80E,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
421,9,129128008,129128008,G,A,exonic,PTPA,.,synonymous SNV,PTPA:NM_178001:exon4:c.G261A:p.Q87Q,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
422,9,129128044,129128044,C,T,exonic,PTPA,.,synonymous SNV,PTPA:NM_178001:exon4:c.C297T:p.R99R,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
423,9,129128065,129128065,G,A,exonic,PTPA,.,synonymous SNV,PTPA:NM_178001:exon4:c.G318A:p.S106S,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
436,9,129129016,129129016,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.C161T:p.T54M,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [36]:
# Filter exonic and non-syn vars
coding_nonsynonymous = PTPA[(PTPA['Func.refGene'] == 'exonic') & (PTPA['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous.count()


Chr              10
Start            10
End              10
Ref              10
Alt              10
                 ..
Otherinfo9648    10
Otherinfo9649    10
Otherinfo9650    10
Otherinfo9651    10
Otherinfo9652    10
Length: 9663, dtype: int64

In [37]:
coding_nonsynonymous.head(10)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo9643,Otherinfo9644,Otherinfo9645,Otherinfo9646,Otherinfo9647,Otherinfo9648,Otherinfo9649,Otherinfo9650,Otherinfo9651,Otherinfo9652
255,9,129120531,129120531,C,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon2:c.C50G:p.P17R,PTPA:NM_...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
436,9,129129016,129129016,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.C161T:p.T54M,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
437,9,129129094,129129094,A,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.A239G:p.Y80C,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
577,9,129134833,129134833,T,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.T412C:p.C138R,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
578,9,129134852,129134852,G,A,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.G431A:p.R144Q,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
579,9,129134882,129134882,A,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
580,9,129134891,129134891,A,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A470G:p.N157S,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
785,9,129142502,129142502,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon8:c.C757T:p.P253S,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
930,9,129147395,129147395,G,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.G816C:p.E272D,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
933,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/1,0/0,0/1,0/0,0/0,0/0,0/1,0/0


In [38]:
print(coding_nonsynonymous)

     Chr      Start        End Ref Alt Func.refGene Gene.refGene  \
255    9  129120531  129120531   C   G       exonic         PTPA   
436    9  129129016  129129016   C   T       exonic         PTPA   
437    9  129129094  129129094   A   G       exonic         PTPA   
577    9  129134833  129134833   T   C       exonic         PTPA   
578    9  129134852  129134852   G   A       exonic         PTPA   
579    9  129134882  129134882   A   C       exonic         PTPA   
580    9  129134891  129134891   A   G       exonic         PTPA   
785    9  129142502  129142502   C   T       exonic         PTPA   
930    9  129147395  129147395   G   C       exonic         PTPA   
933    9  129147457  129147457   C   T       exonic         PTPA   

    GeneDetail.refGene ExonicFunc.refGene  \
255                  .  nonsynonymous SNV   
436                  .  nonsynonymous SNV   
437                  .  nonsynonymous SNV   
577                  .  nonsynonymous SNV   
578                  .  no

In [39]:
coding_nonsynonymous.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo9643,Otherinfo9644,Otherinfo9645,Otherinfo9646,Otherinfo9647,Otherinfo9648,Otherinfo9649,Otherinfo9650,Otherinfo9651,Otherinfo9652
255,9,129120531,129120531,C,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon2:c.C50G:p.P17R,PTPA:NM_...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
436,9,129129016,129129016,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.C161T:p.T54M,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
437,9,129129094,129129094,A,G,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon3:c.A239G:p.Y80C,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
577,9,129134833,129134833,T,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.T412C:p.C138R,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
578,9,129134852,129134852,G,A,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.G431A:p.R144Q,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [40]:
coding = PTPA[PTPA['Func.refGene'] == 'intronic']
coding.count()

Chr              888
Start            888
End              888
Ref              888
Alt              888
                ... 
Otherinfo9648    888
Otherinfo9649    888
Otherinfo9650    888
Otherinfo9651    888
Otherinfo9652    888
Length: 9663, dtype: int64

In [41]:
coding = PTPA[PTPA['Func.refGene'] == 'UTR3']
coding.count()

Chr              52
Start            52
End              52
Ref              52
Alt              52
                 ..
Otherinfo9648    52
Otherinfo9649    52
Otherinfo9650    52
Otherinfo9651    52
Otherinfo9652    52
Length: 9663, dtype: int64

In [42]:
coding = PTPA[PTPA['Func.refGene'] == 'UTR5']
coding.count()

Chr              21
Start            21
End              21
Ref              21
Alt              21
                 ..
Otherinfo9648    21
Otherinfo9649    21
Otherinfo9650    21
Otherinfo9651    21
Otherinfo9652    21
Length: 9663, dtype: int64

## Calculate freq of coding and non-syn vars in cases versus controls

In [43]:
reduced_coding_nonsynonymous = coding_nonsynonymous[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous.txt', sep = '\t', index = False, header= False)

In [44]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

head reduced_coding_nonsynonymous.txt

9	129120531	129120531	PTPA
9	129129016	129129016	PTPA
9	129129094	129129094	PTPA
9	129134833	129134833	PTPA
9	129134852	129134852	PTPA
9	129134882	129134882	PTPA
9	129134891	129134891	PTPA
9	129142502	129142502	PTPA
9	129147395	129147395	PTPA
9	129147457	129147457	PTPA


In [45]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

# calculate freq for ALL variants

/home/jupyter/tools/plink --bfile pheno_PTPA --assoc --out assoc_pheno_PTPA --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to assoc_pheno_PTPA.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PTPA
  --out assoc_pheno_PTPA

14998 MB RAM detected; reserving 7499 MB for main workspace.
986 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to assoc_pheno_PTPA.nosex .
6775 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%

In [46]:
# visualize freq file for ALL variants
all_PTPA_freq = pd.read_csv(f'{WORK_DIR}/assoc_pheno_PTPA.assoc', delim_whitespace=True)
all_PTPA_freq.head()

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
0,9,chr9:129110951:T:C,129110951,C,0.000000,0.000273,T,1.69200,0.1933,0.000
1,9,chr9:129110955:G:C,129110955,C,0.010310,0.007902,G,2.17900,0.1399,1.307
2,9,chr9:129110977:A:C,129110977,C,0.290200,0.289200,A,0.01445,0.9043,1.005
3,9,chr9:129111023:G:C,129111023,C,0.000161,0.000136,G,0.01401,0.9058,1.182
4,9,chr9:129111036:G:C,129111036,C,0.000322,0.000000,G,2.36400,0.1241,NaN


In [47]:
# visualize freq file for ALL variants comparing HC vs PD with p < 0.05 
all_PTPA_freq_p_0_05 = all_PTPA_freq[all_PTPA_freq['P']  <= 0.05]
all_PTPA_freq_p_0_05.count()

CHR      17
SNP      17
BP       17
A1       17
F_A      17
F_U      17
A2       17
CHISQ    17
P        17
OR       11
dtype: int64

In [48]:
all_PTPA_freq_p_0_05.head(17)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
17,9,chr9:129111384:C:A,129111384,A,0.000805,0.002316,C,4.738,0.029510,0.3471
56,9,chr9:129112741:A:T,129112741,T,0.000644,0.000000,A,4.729,0.029650,NaN
72,9,chr9:129113702:G:A,129113702,A,0.000966,0.000136,G,4.488,0.034140,7.0980
135,9,chr9:129116238:G:*,129116238,*,0.000644,0.000000,G,4.729,0.029650,NaN
136,9,chr9:129116246:G:*,129116246,*,0.000644,0.000000,G,4.729,0.029650,NaN
138,9,chr9:129116248:G:*,129116248,*,0.000644,0.000000,G,4.729,0.029650,NaN
141,9,chr9:129116273:C:*,129116273,*,0.000644,0.000000,C,4.729,0.029650,NaN
250,9,chr9:129120424:C:A,129120424,A,0.010140,0.014170,C,4.475,0.034400,0.7131
326,9,chr9:129123630:T:G,129123630,G,0.016910,0.012530,T,4.492,0.034050,1.3550
361,9,chr9:129125432:A:G,129125432,G,0.000000,0.000681,A,4.232,0.039670,0.0000


In [49]:
# visualize freq file for ALL variants only in PD 
all_PTPA_freq_cases_only = all_PTPA_freq[all_PTPA_freq['F_U'] == 0]
all_PTPA_freq_cases_only.count()
all_PTPA_freq_cases_only.head(1000)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
4,9,chr9:129111036:G:C,129111036,C,0.000322,0.0,G,2.364,0.12410,NaN
8,9,chr9:129111181:T:C,129111181,C,0.000161,0.0,T,1.182,0.27690,NaN
11,9,chr9:129111344:A:G,129111344,G,0.000161,0.0,A,1.182,0.27690,NaN
13,9,chr9:129111349:C:T,129111349,T,0.000000,0.0,C,NaN,NaN,NaN
16,9,chr9:129111359:T:G,129111359,T,0.000161,0.0,G,1.182,0.27690,NaN
...,...,...,...,...,...,...,...,...,...,...
971,9,chr9:129148409:C:A,129148409,A,0.000483,0.0,C,3.547,0.05966,NaN
978,9,chr9:129148650:G:A,129148650,A,0.000000,0.0,G,NaN,NaN,NaN
981,9,chr9:129148758:C:T,129148758,T,0.000000,0.0,C,NaN,NaN,NaN
984,9,chr9:129148856:G:C,129148856,G,0.000161,0.0,C,1.182,0.27690,NaN


In [50]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PTPA --extract range reduced_coding_nonsynonymous.txt --assoc --out coding_nonsynonymous_pheno_PTPA --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PTPA.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PTPA
  --ci 0.95
  --extract range reduced_coding_nonsynonymous.txt
  --out coding_nonsynonymous_pheno_PTPA

14998 MB RAM detected; reserving 7499 MB for main workspace.
986 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to coding_nonsynonymous_pheno_PTPA.nosex .
6775 phenotype values loaded from .fam.
--extract range: 976 variants excluded.
--extract range: 10 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%

In [51]:
PTPA_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PTPA.assoc', delim_whitespace=True)
PTPA_freq.head(10)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129120531:C:G,129120531,G,0.000322,0.000000,C,2.36400,0.1241,NaN,NaN,NaN,NaN
1,9,chr9:129129016:C:T,129129016,T,0.000161,0.000000,C,1.18200,0.2769,NaN,NaN,NaN,NaN
2,9,chr9:129129094:A:G,129129094,G,0.002415,0.001771,A,0.67730,0.4105,1.365,0.37930,0.64890,2.870
3,9,chr9:129134833:T:C,129134833,C,0.000000,0.000273,T,1.69200,0.1933,0.000,inf,0.00000,NaN
4,9,chr9:129134852:G:A,129134852,A,0.000000,0.000409,G,2.53900,0.1111,0.000,inf,0.00000,NaN
5,9,chr9:129134882:A:C,129134882,C,0.000161,0.000000,A,1.18200,0.2769,NaN,NaN,NaN,NaN
6,9,chr9:129134891:A:G,129134891,G,0.000000,0.000136,A,0.84610,0.3577,0.000,inf,0.00000,NaN
7,9,chr9:129142502:C:T,129142502,T,0.000322,0.000000,C,2.36400,0.1241,NaN,NaN,NaN,NaN
8,9,chr9:129147395:G:C,129147395,C,0.000161,0.000136,G,0.01401,0.9058,1.182,1.41400,0.07392,18.900
9,9,chr9:129147457:C:T,129147457,T,0.073910,0.071660,C,0.25280,0.6151,1.034,0.06634,0.90790,1.177


In [52]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PTPA --extract range reduced_coding_nonsynonymous.txt --assoc --out coding_nonsynonymous_pheno_PTPA --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PTPA.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_PTPA
  --extract range reduced_coding_nonsynonymous.txt
  --out coding_nonsynonymous_pheno_PTPA

14998 MB RAM detected; reserving 7499 MB for main workspace.
986 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to coding_nonsynonymous_pheno_PTPA.nosex .
6775 phenotype values loaded from .fam.
--extract range: 976 variants excluded.
--extract range: 10 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%

In [53]:
PTPA_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PTPA.assoc.adjusted', delim_whitespace=True)
PTPA_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129134852:G:A,0.1111,0.3224,1,1,0.6920,0.6920,0.4138,1
1,9,chr9:129120531:C:G,0.1241,0.3396,1,1,0.7343,0.6967,0.4138,1
2,9,chr9:129142502:C:T,0.1241,0.3396,1,1,0.7343,0.6967,0.4138,1
3,9,chr9:129134833:T:C,0.1933,0.4191,1,1,0.8833,0.7777,0.4616,1
4,9,chr9:129129016:C:T,0.2769,0.4995,1,1,0.9609,0.8571,0.4616,1
5,9,chr9:129134882:A:C,0.2769,0.4995,1,1,0.9609,0.8571,0.4616,1
6,9,chr9:129134891:A:G,0.3577,0.5678,1,1,0.9880,0.8571,0.5109,1
7,9,chr9:129129094:A:G,0.4105,0.6092,1,1,0.9949,0.8571,0.5132,1
8,9,chr9:129147457:C:T,0.6151,0.7548,1,1,0.9999,0.8571,0.6834,1
9,9,chr9:129147395:G:C,0.9058,0.9414,1,1,1.0000,0.9058,0.9058,1


In [54]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PTPA --extract range reduced_coding_nonsynonymous.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_PTPA
  --extract range reduced_coding_nonsynonymous.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_rare_0.05

14998 MB RAM detected; reserving 7499 MB for main workspace.
986 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to coding_nonsynonymous_rare_0.05.nosex .
6775 phenotype values loaded from .fam.
--extract range: 976 variants excluded.
--extract range: 10 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%

## Explore het carriers

In [55]:
PTPA_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PTPA.raw', delim_whitespace=True)
PTPA_recode.head(10)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
0,BF-1001,BF-1001,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,BF-1002,BF-1002,0,0,0,2,0,0,0,0,0,0,0,0,0,0
2,BF-1003,BF-1003,0,0,0,2,0,0,0,0,0,0,0,0,0,1
3,BF-1004,BF-1004,0,0,0,2,0,0,0,0,0,0,0,0,0,0
4,BF-1005,BF-1005,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,BF-1006,BF-1006,0,0,0,2,0,0,0,0,0,0,0,0,0,0
6,BF-1008,BF-1008,0,0,0,2,0,0,0,0,0,0,0,0,0,0
7,BF-1009,BF-1009,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8,BF-1010,BF-1010,0,0,0,2,0,0,0,0,0,0,0,0,0,0
9,BF-1011,BF-1011,0,0,0,2,0,0,0,0,0,0,0,0,0,0


In [56]:
# Explore heterozygotes for chr9:129129094:A:G_G in cases
PTPA_het_129129094_casos = PTPA_recode[(PTPA_recode['chr9:129129094:A:G_G'] == 1) & (PTPA_recode['PHENOTYPE'] == 2)]
PTPA_het_129129094_casos

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
821,HB-PD_INVPB082TYV,HB-PD_INVPB082TYV,0,0,0,2,0,0,1,0,0,0,0,0,0,0
5991,LC-1420001,LC-1420001,0,0,0,2,0,0,1,0,0,0,0,0,0,0
6013,LC-2130001,LC-2130001,0,0,0,2,0,0,1,0,0,0,0,0,0,0
6034,LC-3390001,LC-3390001,0,0,0,2,0,0,1,0,0,0,0,0,0,0
6113,LC-9450001,LC-9450001,0,0,0,2,0,0,1,0,0,0,0,0,0,0
6755,PD-PDKY896AL6,PD-PDKY896AL6,0,0,0,2,0,0,1,0,0,0,0,0,0,0
7238,PD-PDVT970VTR,PD-PDVT970VTR,0,0,0,2,0,0,1,0,0,0,0,0,0,0
8254,PP-40453,PP-40453,0,0,0,2,0,0,1,0,0,0,0,0,0,0
8335,PP-40916,PP-40916,0,0,0,2,0,0,1,0,0,0,0,0,0,1
8386,PP-41373,PP-41373,0,0,0,2,0,0,1,0,0,0,0,0,0,1


In [57]:
# Explore heterozygotes for chr9:129129094:A:G_G in controles
PTPA_het_129129094_controles = PTPA_recode[(PTPA_recode['chr9:129129094:A:G_G'] == 1) & (PTPA_recode['PHENOTYPE'] == 1)]
PTPA_het_129129094_controles

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
475,HB-PD_INVGG918HA8,HB-PD_INVGG918HA8,0,0,0,1,0,0,1,0,0,0,0,0,0,0
612,HB-PD_INVKC407HM3,HB-PD_INVKC407HM3,0,0,0,1,0,0,1,0,0,0,0,0,0,0
655,HB-PD_INVLA508YEX,HB-PD_INVLA508YEX,0,0,0,1,0,0,1,0,0,0,0,0,0,0
4495,LB-06776,LB-06776,0,0,0,1,0,0,1,0,0,0,0,0,0,0
4876,LB-07231,LB-07231,0,0,0,1,0,0,1,0,0,0,0,0,0,0
4920,LB-07277,LB-07277,0,0,0,1,0,0,1,0,0,0,0,0,0,0
5128,LB-07515,LB-07515,0,0,0,1,0,0,1,0,0,0,0,0,0,0
5769,LC-50005,LC-50005,0,0,0,1,0,0,1,0,0,0,0,0,0,0
5804,LC-140010,LC-140010,0,0,0,1,0,0,1,0,0,0,0,0,0,0
6076,LC-6400001,LC-6400001,0,0,0,1,0,0,1,0,0,0,0,0,0,0


In [117]:
%%bash 
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

grep -e "HB-PD_INVPB082TYV" -e "LC-1420001" -e "LC-2130001" -e "LC-3390001" -e "LC-9450001" -e "PD-PDKY896AL6" -e "PD-PDVT970VTR" -e "PP-40453" -e "PP-40916" -e "PP-41373" -e "PP-41886" -e "PP-42361" -e "PP-58030" -e "PP-59733" -e "PP-71189" COVS.txt > AMP_PD_9:129129094_het_cases.txt
head AMP_PD_9:129129094_het_cases.txt
tail AMP_PD_9:129129094_het_cases.txt

PP-40453	2	2	78
PP-40916	2	2	77
PP-41373	2	2	65
PP-41886	2	2	75
PP-42361	1	2	65
PP-58030	1	2	66
PP-59733	1	2	54
PP-71189	1	2	60
HB-PD_INVPB082TYV	1	2	63
LC-1420001	1	2	68
PP-58030	1	2	66
PP-59733	1	2	54
PP-71189	1	2	60
HB-PD_INVPB082TYV	1	2	63
LC-1420001	1	2	68
LC-2130001	2	2	61
LC-3390001	2	2	76
LC-9450001	1	2	67
PD-PDKY896AL6	2	2	56
PD-PDVT970VTR	2	2	68


In [59]:
%%bash 
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

grep -e "HB-PD_INVGG918HA8" -e "HB-PD_INVKC407HM3" -e "HB-PD_INVLA508YEX" -e "LB-06776" -e "LB-07231" -e "LB-07277" -e "LB-07515" -e "LC-50005" -e "LC-140010" -e "LC-6400001" -e "PD-PDPP720VK5" -e "PP-41310" -e "PP-74303" COVS.txt > AMP_PD_9:129129094_het_controles.txt
head AMP_PD_9:129129094_het_controles.txt
tail AMP_PD_9:129129094_het_controles.txt

HB-PD_INVGG918HA8	1	1	70
HB-PD_INVKC407HM3	1	1	68
HB-PD_INVLA508YEX	1	1	80
LB-06776	1	1	79
LB-07231	2	1	72
LB-07277	1	1	75
LB-07515	1	1	73
LC-140010	2	1	71
LC-50005	2	1	55
LC-6400001	2	1	73
LB-06776	1	1	79
LB-07231	2	1	72
LB-07277	1	1	75
LB-07515	1	1	73
LC-140010	2	1	71
LC-50005	2	1	55
LC-6400001	2	1	73
PD-PDPP720VK5	1	1	56
PP-41310	1	1	79
PP-74303	1	1	50


In [60]:
%%bash 
WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

grep -e "PP-74544" -e "HB-PD_INVDF238ACE" -e "HB-PD_INVLA508YEX" -e "LB-06776" -e "LB-07231" COVS.txt > AMP_PD_9:129120531_het_cases.txt
head AMP_PD_9:129120531_het_cases.txt
tail AMP_PD_9:129120531_het_cases.txt

HB-PD_INVDF238ACE	1	2	60
HB-PD_INVLA508YEX	1	1	80
LB-06776	1	1	79
LB-07231	2	1	72
PP-74544	2	1	68
HB-PD_INVDF238ACE	1	2	60
HB-PD_INVLA508YEX	1	1	80
LB-06776	1	1	79
LB-07231	2	1	72
PP-74544	2	1	68


## Calculate freq of HMZ in cases versus controls

In [61]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PTPA --extract range reduced_coding_nonsynonymous.txt --recode A --out coding_nonsynonymous_pheno_PTPA

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PTPA.log.
Options in effect:
  --bfile pheno_PTPA
  --extract range reduced_coding_nonsynonymous.txt
  --out coding_nonsynonymous_pheno_PTPA
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
986 variants loaded from .bim file.
9640 people (0 males, 0 females, 9640 ambiguous) loaded from .fam.
Ambiguous sex IDs written to coding_nonsynonymous_pheno_PTPA.nosex .
6775 phenotype values loaded from .fam.
--extract range: 976 variants excluded.
--extract range: 10 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 9640 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%

phenotypes to be ignored, use the --allow-no-sex flag.


In [62]:
PTPA_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PTPA.raw', delim_whitespace=True)
PTPA_recode.head(10)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
0,BF-1001,BF-1001,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,BF-1002,BF-1002,0,0,0,2,0,0,0,0,0,0,0,0,0,0
2,BF-1003,BF-1003,0,0,0,2,0,0,0,0,0,0,0,0,0,1
3,BF-1004,BF-1004,0,0,0,2,0,0,0,0,0,0,0,0,0,0
4,BF-1005,BF-1005,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,BF-1006,BF-1006,0,0,0,2,0,0,0,0,0,0,0,0,0,0
6,BF-1008,BF-1008,0,0,0,2,0,0,0,0,0,0,0,0,0,0
7,BF-1009,BF-1009,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8,BF-1010,BF-1010,0,0,0,2,0,0,0,0,0,0,0,0,0,0
9,BF-1011,BF-1011,0,0,0,2,0,0,0,0,0,0,0,0,0,0


In [63]:
# Explora homozigotos para chr9:129147457:C:T_T en casos
PTPA_hom_casos = PTPA_recode[(PTPA_recode['chr9:129147457:C:T_T'] == 2) & (PTPA_recode['PHENOTYPE'] == 2)]
PTPA_hom_casos.head(17)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
99,BF-1163,BF-1163,0,0,0,2,0,0,0,0,0,0,0,0,0,2
918,HB-PD_INVTA736CUG,HB-PD_INVTA736CUG,0,0,0,2,0,0,0,0,0,0,0,0,0,2
1032,HB-PD_INVVT242JK8,HB-PD_INVVT242JK8,0,0,0,2,0,0,0,0,0,0,0,0,0,2
5814,LC-180005,LC-180005,0,0,0,2,0,0,0,0,0,0,0,0,0,2
6655,PD-PDJH196MW5,PD-PDJH196MW5,0,0,0,2,0,0,0,0,0,0,0,0,0,2
6917,PD-PDNL129YR9,PD-PDNL129YR9,0,0,0,2,0,0,0,0,0,0,0,0,0,2
6976,PD-PDPM717VY8,PD-PDPM717VY8,0,0,0,2,0,0,0,0,0,0,0,0,0,2
7067,PD-PDTB425RD6,PD-PDTB425RD6,0,0,0,2,0,0,0,0,0,0,0,0,0,2
7088,PD-PDTG780ZLZ,PD-PDTG780ZLZ,0,0,0,2,0,0,0,0,0,0,0,0,0,2
7141,PD-PDUD605VWP,PD-PDUD605VWP,0,0,0,2,0,0,0,0,0,0,0,0,0,2


In [64]:
PTPA_hom_casos.shape

(17, 16)

In [65]:
# Explore HMZs for chr9:129147457:C:T_T in controls
PTPA_hom_controles = PTPA_recode[(PTPA_recode['chr9:129147457:C:T_T'] == 2) & (PTPA_recode['PHENOTYPE'] == 1)]
PTPA_hom_controles.head(18)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
154,BF-1256,BF-1256,0,0,0,1,0,0,0,0,0,0,0,0,0,2
662,HB-PD_INVLE494MFY,HB-PD_INVLE494MFY,0,0,0,1,0,0,0,0,0,0,0,0,0,2
729,HB-PD_INVML326LKX,HB-PD_INVML326LKX,0,0,0,1,0,0,0,0,0,0,0,0,0,2
752,HB-PD_INVMX828VMK,HB-PD_INVMX828VMK,0,0,0,1,0,0,0,0,0,0,0,0,0,2
1009,HB-PD_INVVF205THQ,HB-PD_INVVF205THQ,0,0,0,1,0,0,0,0,0,0,0,0,0,2
3783,LB-06020,LB-06020,0,0,0,1,0,0,0,0,0,0,0,0,0,2
3874,LB-06113,LB-06113,0,0,0,1,0,0,0,0,0,0,0,0,0,2
4120,LB-06372,LB-06372,0,0,0,1,0,0,0,0,0,0,0,0,0,2
4329,LB-06594,LB-06594,0,0,0,1,0,0,0,0,0,0,0,0,0,2
4476,LB-06753,LB-06753,0,0,0,1,0,0,0,0,0,0,0,0,0,2


In [66]:
PTPA_hom_controles.shape

(18, 16)

In [67]:
'Calcula frecuencia frente al N total de controles: {:f}'.format(18/3670)

'Calcula frecuencia frente al N total de controles: 0.004905'

In [68]:
# Explora homozigotos para chr9:129134852:G:A_A en casos
PTPA_hom_casos = PTPA_recode[(PTPA_recode['chr9:129134852:G:A_A'] == 2) & (PTPA_recode['PHENOTYPE'] == 2)]
PTPA_hom_casos.head(17)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T


In [69]:
PTPA_hom_casos.shape

(0, 16)

In [70]:
# Explore HMZs for chr9:129134852:G:A_A in controls
PTPA_hom_controles = PTPA_recode[(PTPA_recode['chr9:129134852:G:A_A'] == 2) & (PTPA_recode['PHENOTYPE'] == 1)]
PTPA_hom_controles.head(18)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
5354,LB-07787,LB-07787,0,0,0,1,0,0,0,0,2,0,0,0,0,0


In [71]:
PTPA_hom_controles.shape

(1, 16)

## COMP HET

In [72]:
# Identifying individuals het for the alternative allele
het = PTPA_recode[(PTPA_recode['chr9:129147457:C:T_T'] == 1) | (PTPA_recode['chr9:129147395:G:C_C'] == 1) | (PTPA_recode['chr9:129142502:C:T_T'] == 1) | (PTPA_recode['chr9:129134891:A:G_G'] == 1) | (PTPA_recode['chr9:129134882:A:C_C'] == 1) | (PTPA_recode['chr9:129134852:G:A_A'] == 1) | (PTPA_recode['chr9:129134833:T:C_C'] == 1) | (PTPA_recode['chr9:129129094:A:G_G'] == 1) | (PTPA_recode['chr9:129129016:C:T_T'] == 1) | (PTPA_recode['chr9:129120531:C:G_G'] == 1)]
het = het.dropna()
print(het.shape)

(1324, 16)


In [73]:
het.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
2,BF-1003,BF-1003,0,0,0,2,0,0,0,0,0,0,0,0,0,1
7,BF-1009,BF-1009,0,0,0,1,0,0,0,0,0,0,0,0,0,1
11,BF-1013,BF-1013,0,0,0,1,0,0,0,0,0,0,0,0,0,1
14,BF-1016,BF-1016,0,0,0,2,0,0,0,0,0,0,0,0,0,1
26,BF-1046,BF-1046,0,0,0,2,0,0,0,0,0,0,0,0,0,1


In [74]:
het['compound'] = het['chr9:129147457:C:T_T'] + het['chr9:129147395:G:C_C'] + het['chr9:129142502:C:T_T'] + het['chr9:129134891:A:G_G'] + het['chr9:129134882:A:C_C'] + het['chr9:129134852:G:A_A'] + het['chr9:129134833:T:C_C'] + het['chr9:129129094:A:G_G'] + het['chr9:129129016:C:T_T'] + het['chr9:129120531:C:G_G']
het['compound'].value_counts()

1    1321
2       3
Name: compound, dtype: int64

In [75]:
comp_het_info_cases = het[(het['PHENOTYPE'] == 2) & (het['compound'] == 2)]
print(comp_het_info_cases.shape)

(2, 17)


In [76]:
comp_het_info_control = het[(het['PHENOTYPE'] == 1) & (het['compound'] == 1)]
print(comp_het_info_control.shape)

(508, 17)


In [77]:
comp_het_info_cases.to_csv('/home/jupyter/PTPA/comp_het_info_cases.txt', sep = '\t', index=True)
comp_het_info_cases.head(508)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T,compound
8335,PP-40916,PP-40916,0,0,0,2,0,0,1,0,0,0,0,0,0,1,2
8386,PP-41373,PP-41373,0,0,0,2,0,0,1,0,0,0,0,0,0,1,2


## Hetero in R

In [78]:
!pip install rpy2
%load_ext rpy2.ipython

In [79]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [118]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [81]:
# Explore heterozygotes for chr9:129147457:C:T_T in cases
PTPA_het_129147457_casos = PTPA_recode[(PTPA_recode['chr9:129147457:C:T_T'] == 1) & (PTPA_recode['PHENOTYPE'] == 2)]
PTPA_het_129147457_casos

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
2,BF-1003,BF-1003,0,0,0,2,0,0,0,0,0,0,0,0,0,1
14,BF-1016,BF-1016,0,0,0,2,0,0,0,0,0,0,0,0,0,1
26,BF-1046,BF-1046,0,0,0,2,0,0,0,0,0,0,0,0,0,1
40,BF-1065,BF-1065,0,0,0,2,0,0,0,0,0,0,0,0,0,1
46,BF-1079,BF-1079,0,0,0,2,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9613,SY-PDXR916GN6,SY-PDXR916GN6,0,0,0,2,0,0,0,0,0,0,0,0,0,1
9615,SY-PDXV507VWG,SY-PDXV507VWG,0,0,0,2,0,0,0,0,0,0,0,0,0,1
9632,SY-PDZK862NFW,SY-PDZK862NFW,0,0,0,2,0,0,0,0,0,0,0,0,0,1
9634,SY-PDZU300RYJ,SY-PDZU300RYJ,0,0,0,2,0,0,0,0,0,0,0,0,0,1


In [82]:
PTPA_het_129147457_casos.shape

(425, 16)

In [138]:
PTPA_het_129147457_casos.to_csv('PTPA_het_129147457_casos.csv')

In [139]:
temp = pd.read_csv('PTPA_het_129147457_casos.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
0,2,BF-1003,BF-1003,0,0,0,2,0,0,0,0,0,0,0,0,0,1
1,14,BF-1016,BF-1016,0,0,0,2,0,0,0,0,0,0,0,0,0,1
2,26,BF-1046,BF-1046,0,0,0,2,0,0,0,0,0,0,0,0,0,1
3,40,BF-1065,BF-1065,0,0,0,2,0,0,0,0,0,0,0,0,0,1
4,46,BF-1079,BF-1079,0,0,0,2,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,9613,SY-PDXR916GN6,SY-PDXR916GN6,0,0,0,2,0,0,0,0,0,0,0,0,0,1
421,9615,SY-PDXV507VWG,SY-PDXV507VWG,0,0,0,2,0,0,0,0,0,0,0,0,0,1
422,9632,SY-PDZK862NFW,SY-PDZK862NFW,0,0,0,2,0,0,0,0,0,0,0,0,0,1
423,9634,SY-PDZU300RYJ,SY-PDZU300RYJ,0,0,0,2,0,0,0,0,0,0,0,0,0,1


In [140]:
%%bash

WORK_DIR='/home/jupyter/PTPA/'
cd $WORK_DIR
head COVS.txt

ID	SEX	PHENO	AGE_BASELINE
BF-1002	2	2	66
BF-1003	1	2	61
BF-1004	1	2	62
BF-1006	1	2	59
BF-1008	2	2	69
BF-1010	1	2	65
BF-1011	1	2	70
BF-1015	1	2	61
BF-1016	2	2	71


In [142]:
%%R
setwd('/home/jupyter/PTPA/')
library(data.table)
covs <- fread("COVS.txt", header =T)
covs

            ID SEX PHENO AGE_BASELINE
   1:  BF-1002   2     2           66
   2:  BF-1003   1     2           61
   3:  BF-1004   1     2           62
   4:  BF-1006   1     2           59
   5:  BF-1008   2     2           69
  ---                                
7835: PP-75512   2     1           51
7836: PP-75520   1     1           63
7837: PP-75547   1     1           52
7838: PP-75550   1     1           69
7839: PP-92834   1     1           66


In [143]:
%%R
colnames(covs)[1]  <- "FID"  
covs

           FID SEX PHENO AGE_BASELINE
   1:  BF-1002   2     2           66
   2:  BF-1003   1     2           61
   3:  BF-1004   1     2           62
   4:  BF-1006   1     2           59
   5:  BF-1008   2     2           69
  ---                                
7835: PP-75512   2     1           51
7836: PP-75520   1     1           63
7837: PP-75547   1     1           52
7838: PP-75550   1     1           69
7839: PP-92834   1     1           66


In [150]:
%%R
setwd('/home/jupyter/PTPA/')
library(data.table)
PTPA_het_129147457_casos <- fread("PTPA_het_129147457_casos.csv", header =T)
PTPA_het_129147457_casos
temp <- merge(covs, PTPA_het_129147457_casos, by="FID")
head(temp)
write.table(temp, file = "PTPA_het_129147457_casos_extracted.txt", quote = F, row.names = F, sep = "\t")
PTPA_het_129147457_casos_extracted <- fread("PTPA_het_129147457_casos_extracted.txt", header =T)
PTPA_het_129147457_casos_extracted

               FID SEX.x PHENO AGE_BASELINE   V1           IID PAT MAT SEX.y
  1:       BF-1003     1     2           61    2       BF-1003   0   0     0
  2:       BF-1016     2     2           71   14       BF-1016   0   0     0
  3:       BF-1046     2     2           65   26       BF-1046   0   0     0
  4:       BF-1065     1     2           68   40       BF-1065   0   0     0
  5:       BF-1079     1     2           60   46       BF-1079   0   0     0
 ---                                                                        
421: SY-PDXR916GN6     1     2           68 9613 SY-PDXR916GN6   0   0     0
422: SY-PDXV507VWG     1     2           68 9615 SY-PDXV507VWG   0   0     0
423: SY-PDZK862NFW     1     2           63 9632 SY-PDZK862NFW   0   0     0
424: SY-PDZU300RYJ     1     2           64 9634 SY-PDZU300RYJ   0   0     0
425: SY-PDZX943HWN     1     2           46 9638 SY-PDZX943HWN   0   0     0
     PHENOTYPE chr9:129120531:C:G_G chr9:129129016:C:T_T chr9:129129094:A:G_

In [87]:
# Explore heterozygotes for chr9:129147457:C:T_T in controles
PTPA_het_129147457_controles = PTPA_recode[(PTPA_recode['chr9:129147457:C:T_T'] == 1) & (PTPA_recode['PHENOTYPE'] == 1)]
PTPA_het_129147457_controles

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
7,BF-1009,BF-1009,0,0,0,1,0,0,0,0,0,0,0,0,0,1
11,BF-1013,BF-1013,0,0,0,1,0,0,0,0,0,0,0,0,0,1
27,BF-1047,BF-1047,0,0,0,1,0,0,0,0,0,0,0,0,0,1
34,BF-1058,BF-1058,0,0,0,1,0,0,0,0,0,0,0,0,0,1
59,BF-1097,BF-1097,0,0,0,1,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8892,PP-59053,PP-59053,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8958,PP-65008,PP-65008,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8962,PP-70087,PP-70087,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8991,PP-71236,PP-71236,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [105]:
PTPA_het_129147457_controles.to_csv('PTPA_het_129147457_controles.csv')

In [106]:
temp = pd.read_csv('PTPA_het_129147457_controles.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129120531:C:G_G,chr9:129129016:C:T_T,chr9:129129094:A:G_G,chr9:129134833:T:C_C,chr9:129134852:G:A_A,chr9:129134882:A:C_C,chr9:129134891:A:G_G,chr9:129142502:C:T_T,chr9:129147395:G:C_C,chr9:129147457:C:T_T
0,7,BF-1009,BF-1009,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1,11,BF-1013,BF-1013,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,27,BF-1047,BF-1047,0,0,0,1,0,0,0,0,0,0,0,0,0,1
3,34,BF-1058,BF-1058,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,59,BF-1097,BF-1097,0,0,0,1,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,8892,PP-59053,PP-59053,0,0,0,1,0,0,0,0,0,0,0,0,0,1
486,8958,PP-65008,PP-65008,0,0,0,1,0,0,0,0,0,0,0,0,0,1
487,8962,PP-70087,PP-70087,0,0,0,1,0,0,0,0,0,0,0,0,0,1
488,8991,PP-71236,PP-71236,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [151]:
%%R
setwd('/home/jupyter/PTPA/')
library(data.table)
PTPA_het_129147457_controles <- fread("PTPA_het_129147457_controles.csv", header =T)
PTPA_het_129147457_controles
temp <- merge(covs, PTPA_het_129147457_controles, by="FID")
head(temp)
write.table(temp, file = "PTPA_het_129147457_controles_extracted.txt", quote = F, row.names = F, sep = "\t")
PTPA_het_129147457_controles_extracted <- fread("PTPA_het_129147457_controles_extracted.txt", header =T)
PTPA_het_129147457_controles_extracted

          FID SEX.x PHENO AGE_BASELINE   V1      IID PAT MAT SEX.y PHENOTYPE
  1:  BF-1009     1     1           73    7  BF-1009   0   0     0         1
  2:  BF-1013     1     1           81   11  BF-1013   0   0     0         1
  3:  BF-1047     1     1           56   27  BF-1047   0   0     0         1
  4:  BF-1058     2     1           70   34  BF-1058   0   0     0         1
  5:  BF-1097     2     1           68   59  BF-1097   0   0     0         1
 ---                                                                        
486: PP-59053     2     1           63 8892 PP-59053   0   0     0         1
487: PP-65008     2     1           69 8958 PP-65008   0   0     0         1
488: PP-70087     2     1           63 8962 PP-70087   0   0     0         1
489: PP-71236     1     1           71 8991 PP-71236   0   0     0         1
490: PP-73397     1     1           53 9013 PP-73397   0   0     0         1
     chr9:129120531:C:G_G chr9:129129016:C:T_T chr9:129129094:A:G_G
  1:    

## Save out results..!

In [152]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-02e7cd23 cp -r /home/jupyter/PTPA/ gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0


Copying file:///home/jupyter/PTPA/reduced_coding_nonsynonymous.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PTPA/temp.fam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PTPA/temp [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PTPA/pheno_PTPA.annovar.hg38_multianno.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PTPA/temp.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PTPA/assoc_pheno_PTPA.assoc [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PTPA/coding_nonsynonymous_pheno_PTPA.bim [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PTPA/PTPA_het_129147457_casos.csv [Content-Type=text/csv]...
Copying file:///home/jupyter/PTPA/pheno_PTPA.nosex [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PTPA/pheno_PTPA.annovar.hg38_multianno.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/PTPA/coding_nonsynonym